In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import user_knn
from lenskit.algorithms import Recommender

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kirillgugunishvili/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Scenario/ context:
# organisation helping with applications
# offices around the world
# limited resources so tries to group people as best it can (i.e. according to degree)
#   but sometimes not possible so just groups however comes into single session 
# people sign up for a session and can be grouped with people most similar to them 
# recommender engine is needed for organisers of the company to ensure they are able to conduct 
#   workshops that intrest the majority of group members in a session

# Procedure:
#   Separate people by education level: high school, Non, Graduates
#   Separate each group by state
#   In addition separate Graduates group by major (same major)
#   Limit group size: 20-30

# load intial data
user_data = pd.read_csv("dataset/users.tsv", sep='\t')
display(user_data.head(10))


,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
5,131,1,Train,Houston,TX,US,77077,Bachelor's,Finance,1998-05-01 00:00:00,3,14.0,NaN,No,0
6,162,1,Train,Long Beach,CA,US,90807,Master's,I/O Psychology,2012-05-01 00:00:00,10,25.0,No,No,0
7,178,1,Train,Greenville,SC,US,29609,High School,Not Applicable,NaN,6,35.0,No,Yes,4
8,203,1,Train,Colchester,VT,US,05446,Master's,Burlington,NaN,4,3.0,Yes,No,0
9,344,1,Train,Newport News,VA,US,23601,High School,Not Applicable,2007-01-01 00:00:00,3,7.0,Yes,No,0


In [12]:
# extract state list
state_col = user_data[["State"]].drop_duplicates().reset_index(drop=True)
display(state_col)


,State
0,CA
1,NJ
2,NY
3,LA
4,TX
...,...
217,Zaragoza
218,Rheinland-Pfalz
219,Durham
220,Auvergne


In [4]:
# extract high schoolers
undergrads_data = user_data[user_data["DegreeType"]=="High School"]
undergrads_data = undergrads_data.reset_index(drop=True).fillna("Not Applicable")
display(undergrads_data)

# extract non high schoolers
grads_data = user_data.fillna("Not Applicable")
grads_data = grads_data[(grads_data["DegreeType"]!="High School") & (grads_data["DegreeType"]!="Not Applicable")]
grads_data = grads_data.reset_index(drop=True)
display(grads_data)

# extract no education
noedu_data = user_data.fillna("Not Applicable")
noedu_data = noedu_data[noedu_data["DegreeType"]=="Not Applicable"]
noedu_data = noedu_data.reset_index(drop=True)
display(noedu_data)


,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,Not Applicable,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
2,178,1,Train,Greenville,SC,US,29609,High School,Not Applicable,Not Applicable,6,35.0,No,Yes,4
3,344,1,Train,Newport News,VA,US,23601,High School,Not Applicable,2007-01-01 00:00:00,3,7.0,Yes,No,0
4,496,1,Train,Easley,SC,US,29640,High School,Not Applicable,1987-01-01 00:00:00,10,21.0,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93300,1471765,7,Train,Kennesaw,GA,US,30144,High School,Not Applicable,2012-06-01 00:00:00,1,2.0,No,No,0
93301,1471878,7,Train,Orangeburg,SC,US,29115,High School,Not Applicable,1999-01-01 00:00:00,4,3.0,No,No,0
93302,1471997,7,Train,Baltimore,MD,US,21222,High School,Math,2004-06-01 00:00:00,3,5.0,No,Yes,45
93303,1472042,7,Train,Saint Clair Shores,MI,US,48081,High School,Not Applicable,1970-01-01 00:00:00,7,32.0,Yes,No,0


,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
1,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
2,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
3,131,1,Train,Houston,TX,US,77077,Bachelor's,Finance,1998-05-01 00:00:00,3,14.0,Not Applicable,No,0
4,162,1,Train,Long Beach,CA,US,90807,Master's,I/O Psychology,2012-05-01 00:00:00,10,25.0,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196245,1471622,7,Train,Jersey City,NJ,US,07307,Bachelor's,Psychology,2012-06-01 00:00:00,4,8.0,Yes,No,0
196246,1471633,7,Train,Sacramento,CA,US,95823,Bachelor's,Deaf Education,1985-01-01 00:00:00,9,14.0,Yes,Yes,30
196247,1471828,7,Train,Atlanta,GA,US,30312,PhD,Not Applicable,2006-05-01 00:00:00,3,6.0,Yes,No,0
196248,1472058,7,Train,Maspeth,NY,US,11378,Associate's,Not Applicable,2001-01-01 00:00:00,2,8.0,Yes,Yes,3


,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,411,1,Train,Lutz,FL,US,33559,Not Applicable,Not Applicable,Not Applicable,3,11.0,Yes,Yes,1
1,640,1,Train,Cooper City,FL,US,33330,Not Applicable,Not Applicable,Not Applicable,4,34.0,Not Applicable,No,0
2,1113,1,Train,Ramsey,NJ,US,07446,Not Applicable,Not Applicable,2011-10-01 00:00:00,2,1.0,No,No,0
3,1123,1,Train,Jacksonville,FL,US,32256,Not Applicable,Civil Engineering and Architecture,2012-12-01 00:00:00,3,9.0,Yes,Yes,100
4,1226,1,Train,Cincinnati,OH,US,45245,Not Applicable,Licensed Practical/Vocational Nurse Training (...,1988-12-01 00:00:00,8,23.0,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100148,1471719,7,Train,Baltimore,MD,US,21216,Not Applicable,Customer service,2008-01-01 00:00:00,0,Not Applicable,No,No,0
100149,1471870,7,Train,Lubbock,TX,US,79412,Not Applicable,Criminal Justice,2003-01-01 00:00:00,5,19.0,No,No,0
100150,1471877,7,Train,Greer,SC,US,29650,Not Applicable,Business Administration,1988-01-01 00:00:00,9,30.0,Not Applicable,No,0
100151,1471901,7,Train,Port Clinton,OH,US,43452,Not Applicable,Not Applicable,Not Applicable,7,25.0,No,No,0


In [5]:
# Count how many jobs
counts = grads_data.groupby(["Major"]).size().reset_index(name='count').sort_values(by='count', ascending=False)
display(counts.head(5))

,Major,count
25535,Not Applicable,20521
5010,Business Administration,10725
384,Accounting,6672
6178,Business Management,4801
27972,Psychology,4323


In [19]:
# createGrps(data_users, minP, maxP, grpNum, seed)

minP = 20
maxP = 20
size = 20
grpNum = 20
seed = 1234567

In [13]:
# unique majors
major_col = grads_data[["Major"]].drop_duplicates().reset_index(drop=True)
display(major_col.head(5))

,Major
0,Anthropology
1,Journalism
2,Agricultural Business
3,Finance
4,I/O Psychology


,Users


In [70]:
import random 
from util import selectUsersRand

groups = pd.DataFrame([], columns=["Users"])
display(groups)

# extract unique column of states
state_col_0 = noedu_data[["State"]].drop_duplicates().reset_index(drop=True)

state_col_1 = undergrads_data[["State"]].drop_duplicates().reset_index(drop=True)

state_col_2 = grads_data[["State"]].drop_duplicates().reset_index(drop=True)

# extract unqiue column of majors 
major_col = grads_data[["Major"]].drop_duplicates().reset_index(drop=True)
majorLen = len(major_col)

# set randomness seed
random.seed(seed)
# start generating groups of user ids
grpCount = 0
while grpCount < grpNum:
    # choose which of three large groups to access
    grp = random.randint(0,2)
    if grp == 0: # no education group
        working_data = noedu_data
        state_col = state_col_0
    elif grp == 1: # high school group
        working_data = undergrads_data
        state_col = state_col_1
    else: # (grp == 2) # graduate group
        working_data = grads_data
        state_col = state_col_2
    # reset major column

    # get length of entries
    stateLen = len(state_col)

    # After selecting one of three large groups split by random state
    rndState = random.randint(0,stateLen-1)
    stt = state_col["State"].loc[rndState]
    working_data = working_data[working_data["State"]==stt]

    # Then if graduate group also split by random major
    if grp == 2:
        rndMaj = random.randint(0,majorLen-1)
        maj = major_col["Major"].loc[rndMaj]
        working_data = working_data[working_data["State"]==maj]
    
    # select <size> (e.g. 20) random users
    if (len(working_data)>0):
        edited_data = working_data
        cntr = 0
        users = np.array([])
        #display(edited_data)
        while cntr < size:
            id = random.choice(np.array(edited_data["UserID"]))
            users = np.append(users, id)
            edited_data = edited_data[edited_data["UserID"]!=id]
            cntr+=1
            if(len(edited_data)==0): break
        
        groups.loc[grpCount] = [users]

        # form next group
        grpCount+=1


,Users


In [71]:
display(groups)

,Users
0,"[379846.0, 877836.0, 1237855.0, 31806.0, 53279..."
1,[384122.0]
2,"[945852.0, 1266328.0]"
3,[266263.0]
4,"[460672.0, 523957.0, 418380.0, 621454.0]"
5,[93235.0]
6,[601513.0]
7,"[360057.0, 811679.0, 1158472.0, 179776.0]"
8,"[1218893.0, 923308.0, 698182.0, 1195195.0, 411..."
9,[866505.0]
